In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [7]:
data_file = '/Users/pdvsmac/Dropbox/Geekbrains/Рекомендательные системы_20200705/Урок 2. Бейзлайны и детерминированные алгоритмы item-item/\
retail_train.csv'
data = pd.read_csv(data_file) 
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [8]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [36]:
def weighted_random_recommendation(items, items_weights, n=5):
    # your_code
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, p=items_weights, replace=False)    
    
    return recs.tolist()

In [56]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
# берем где продажи > 0
popularity_positive = popularity[popularity['sales_value'] > 0]
# берем логариф продаж
popularity_positive['sales_sum_log'] = np.log(popularity_positive['sales_value'])
# сделаем нормализацию 0-1
popularity_positive['sales_sum_log_norm']=((popularity_positive['sales_sum_log']-popularity_positive['sales_sum_log'].min())/\
                                           (popularity_positive['sales_sum_log'].max()-popularity_positive['sales_sum_log'].min()))

popularity_positive.head()


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,item_id,sales_value,sales_sum_log,sales_sum_log_norm
0,25671,20.94,3.041661,0.790111
1,26081,0.99,-0.010050,0.726152
2,26093,1.59,0.463734,0.736082
3,26190,1.54,0.431782,0.735412
4,26355,1.98,0.683097,0.740679


In [61]:
%%time

popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
# берем где продажи > 0
popularity_positive = popularity[popularity['sales_value'] > 0]
# берем логариф продаж
popularity_positive['sales_sum_log'] = np.log(popularity_positive['sales_value'])
# сделаем нормализацию 0-1
popularity_positive['sales_sum_log_norm'] = ((popularity_positive['sales_sum_log']-popularity_positive['sales_sum_log'].min())/\
                                           (popularity_positive['sales_sum_log'].max()-popularity_positive['sales_sum_log'].min()))

# сделаем переведем в вероятность 0-1
popularity_positive['sales_sum_log_prob'] = popularity_positive['sales_sum_log_norm'] / popularity_positive['sales_sum_log_norm'].sum()


items = popularity_positive['item_id']
items_weights = popularity_positive['sales_sum_log_prob']

popular_recs = weighted_random_recommendation(items, items_weights, n=5)

# result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
# result.head(2)

CPU times: user 281 ms, sys: 13.2 ms, total: 294 ms
Wall time: 293 ms


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing

In [62]:
popular_recs

[1024139, 1098172, 9552767, 1119357, 863186]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [421]:
result = pd.read_csv('predictions_basic.csv', sep=',', engine='python')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[986727, 1003178, 852065, 861603, 9837871]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1000112, 3218130, 996892, 883606, 43848]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [422]:
# не разобрался как правильно csv прочитать, поэтому пришлось регулярки задействовать

In [423]:
# str to list
def str_to_list(x):
    return list(map(int, x.replace('[','').replace(']','').replace('    ',' ').replace('   ',' ').replace('  ',' ').replace('\n',' ').split(',')))

In [424]:
# Преобразуем result['actual'] to list of int
result['actual'] = result['actual'].apply(lambda x: \
                       list(map(int, x.replace('[','').replace(']','').replace('    ',' ').replace('   ',' ').replace('  ',' ').replace('\n',' ').split())))

# Остальные стлобцы тоже
result['random_recommendation'] = result['random_recommendation'].apply(lambda x: str_to_list(x))
result['popular_recommendation'] = result['popular_recommendation'].apply(lambda x: str_to_list(x))
result['itemitem'] = result['itemitem'].apply(lambda x: str_to_list(x))
result['cosine'] = result['cosine'].apply(lambda x: str_to_list(x))
result['tfidf'] = result['tfidf'].apply(lambda x: str_to_list(x))
result['own_purchases'] = result['own_purchases'].apply(lambda x: str_to_list(x))

In [425]:
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[986727, 1003178, 852065, 861603, 9837871]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1000112, 3218130, 996892, 883606, 43848]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[13073054, 994262, 1347676, 1108792, 933664]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1098066, 826249, 1106523, 923746, 1058997]","[1098066, 826249, 860776, 854852, 1068719]","[1098066, 826249, 860776, 1068719, 916122]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[712280, 1088433, 10355942, 2201446, 829191]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1098066, 840361, 883404, 916122]","[981760, 1098066, 883404, 1004906, 859075]","[981760, 883404, 1098066, 859075, 916122]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1101667, 1107585, 6391557, 990771, 15830886]","[6534178, 6533889, 1029743, 6534166, 1082185]","[904360, 13115903, 13189726, 13190294, 15596515]","[904360, 5588666, 1096036, 979707, 1013321]","[904360, 1096036, 5588666, 979707, 1013321]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [426]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

# your_code

#list[colname: precisin@k]


In [427]:
result['precision_random_recommendation'] = result.apply(lambda x: \
                                                         precision_at_k(x['random_recommendation'], x['actual'], k=5), axis=1)
result['precision_popular_recommendation'] = result.apply(lambda x: \
                                                         precision_at_k(x['popular_recommendation'], x['actual'], k=5), axis=1)
result['precision_itemitem'] = result.apply(lambda x: \
                                                         precision_at_k(x['itemitem'], x['actual'], k=5), axis=1)
result['precision_cosine'] = result.apply(lambda x: \
                                                         precision_at_k(x['cosine'], x['actual'], k=5), axis=1)
result['precision_tfidf'] = result.apply(lambda x: \
                                                         precision_at_k(x['tfidf'], x['actual'], k=5), axis=1)
result['precision_own_purchases'] = result.apply(lambda x: \
                                                         precision_at_k(x['own_purchases'], x['actual'], k=5), axis=1)

In [428]:
{'precision_random_recommendation': result['precision_random_recommendation'].mean(),
'precision_popular_recommendation': result['precision_popular_recommendation'].mean(),
'precision_itemitem': result['precision_itemitem'].mean(),
'precision_cosine': result['precision_cosine'].mean(),
'precision_tfidf': result['precision_tfidf'].mean(),
'precision_own_purchases': result['precision_own_purchases'].mean()}

{'precision_random_recommendation': 0.00039177277179236047,
 'precision_popular_recommendation': 0.15523996082272082,
 'precision_itemitem': 0.033594515181195064,
 'precision_cosine': 0.03525954946131262,
 'precision_tfidf': 0.03614103819784544,
 'precision_own_purchases': 0.17969311132876015}

Как и следовало ожидать рекомендации на основе попоулярных товаров и собственных покупок показувыют существенно лучший результат

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code